In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import json
import os
from tqdm import tqdm

# --- Paths (make sure these are correct) ---
VIDEO_SOURCE_DIR = r"D:\WLASL DataSet\WLASL FULL\videos"
SPLIT_FILE_PATH = r"D:\WLASL DataSet\WLASL FULL\nslt_300.json"
OUTPUT_NPZ_PATH = r'D:\New Projects\Landmarks_553() for 300 Classes\Full_Landmarks_300_553_3D.npz'

# --- Parameters ---
SAVE_CHECKPOINT_EVERY_N_VIDEOS = 250
TOTAL_LANDMARKS = 553 # Correct total

# --- Landmark extraction function (now accepts the model as an argument) ---
def extract_landmarks(video_path, holistic_model):
    """Extracts 553 landmarks from a video file using a pre-initialized model."""
    video_landmarks = []
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None
        
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False # Performance optimization
            results = holistic_model.process(image)
            image.flags.writeable = True

            frame_data = np.zeros((TOTAL_LANDMARKS, 3), dtype=np.float32)

            def extract_and_fill(landmark_list, start_idx, num_landmarks):
                if landmark_list:
                    for i, landmark in enumerate(landmark_list.landmark):
                        if i < num_landmarks:
                            frame_data[start_idx + i] = [landmark.x, landmark.y, landmark.z]

            start_idx = 0
            extract_and_fill(results.pose_landmarks, start_idx, 33)
            start_idx += 33
            # --- FIX IS HERE ---
            extract_and_fill(results.face_landmarks, start_idx, 478) # Corrected from 468 to 478
            start_idx += 478
            # --- END OF FIX ---
            extract_and_fill(results.left_hand_landmarks, start_idx, 21)
            start_idx += 21
            extract_and_fill(results.right_hand_landmarks, start_idx, 21)
            
            video_landmarks.append(frame_data)
            
    finally:
        cap.release()
        
    if not video_landmarks: return None
    return np.array(video_landmarks, dtype=np.float32)
# --- Main processing logic ---
def main():
    # Load the list of video IDs to process
    with open(SPLIT_FILE_PATH, 'r') as f:
        # --- FIX IS HERE: Pass the file object 'f' to json.load() ---
        data = json.load(f)
        video_ids = list(data.keys())
    
    video_paths_to_process = [(vid, os.path.join(VIDEO_SOURCE_DIR, f'{vid}.mp4')) for vid in video_ids]
    
    existing_landmarks = {}
    
    # --- OPTIMIZATION: Initialize model ONCE ---
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    try:
        for i, (video_id, video_path) in enumerate(tqdm(video_paths_to_process, desc="Extracting Landmarks")):
            if not os.path.exists(video_path):
                continue

            landmarks = extract_landmarks(video_path, holistic)
            if landmarks is not None:
                existing_landmarks[video_id] = landmarks

            # Save periodically
            if (i > 0) and (i + 1) % SAVE_CHECKPOINT_EVERY_N_VIDEOS == 0:
                print(f"\n💾 Saving checkpoint at {len(existing_landmarks)} videos...")
                np.savez_compressed(OUTPUT_NPZ_PATH, **existing_landmarks)
    finally:
        # --- Make sure to close the model ---
        holistic.close() 

    # --- Final save ---
    print(f"\n✅ All done! Extracted landmarks for {len(existing_landmarks)} videos.")
    np.savez_compressed(OUTPUT_NPZ_PATH, **existing_landmarks)
    print(f"🎉 Landmarks saved at: {OUTPUT_NPZ_PATH}")


In [2]:

# Add this to run your main function
if __name__ == '__main__':
    main()


Extracting Landmarks:   5%|▍         | 249/5118 [24:45<4:19:49,  3.20s/it] 


💾 Saving checkpoint at 250 videos...


Extracting Landmarks:  10%|▉         | 499/5118 [39:52<3:58:44,  3.10s/it]


💾 Saving checkpoint at 500 videos...


Extracting Landmarks:  15%|█▍        | 749/5118 [53:48<4:07:44,  3.40s/it]

...


💾 Saving checkpoint at 4750 videos...


Extracting Landmarks:  98%|█████████▊| 4999/5118 [7:33:11<06:39,  3.35s/it]  


💾 Saving checkpoint at 5000 videos...


Extracting Landmarks: 100%|██████████| 5118/5118 [7:41:41<00:00,  5.41s/it]  



✅ All done! Extracted landmarks for 5118 videos.
🎉 Landmarks saved at: D:\New Projects\Landmarks_553() for 300 Classes\Full_Landmarks_300_553_3D.npz
